In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
This script creates an array containing energy in kWh
for a certain PV size based on ERA5-land data in a single coordinate

Created on Monday July  29 14:17:50 2019

@author: luis@refuel
"""

import xarray as xr
import pvlib
import pandas as pd
import numpy as np
import pvlib
from pvlib.pvsystem import PVSystem
from pvlib.location import Location
from pvlib.modelchain import ModelChain





The first step is to transform the cumulated values in J/sqm to instantaneous radiation values in W/sqm

load Specifications from standard/declared modules and inversters

In [6]:
# load the module and inverter specifications
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')

cec_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')

sandia_module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']

cec_inverter = cec_inverters['ABB__MICRO_0_25_I_OUTD_US_208_208V__CEC_2014_']

define the location and the system parameters

In [7]:
location = Location(latitude=32.2, longitude=-110.9)

system = PVSystem(surface_tilt=20, surface_azimuth=2,
                  module_parameters=sandia_module,
                  inverter_parameters=cec_inverter)

mc = ModelChain(system, location)

In [8]:
print(mc)

ModelChain: 
  name: None
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: sapm
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses


##to do -> modify the example escript changing the time step by a time series

In [ ]:
weather = pd.DataFrame([[ghi_era5l, dni_era5l, dhi_era5l, temp_era5l, wind_speed_era5l]],
                       columns=['ghi', 'dni', 'dhi', 'temp_air', 'wind_speed'],
                       index=[pd.Timestamp('20170401 1200', tz='UTC')])

mc.run_model(times=weather.index, weather=weather)

In [ ]:
#separate the ssrd into direct and diffuse radiation 
 

def calc_pv_output(year, input_dir_radiation, input_dif_radiation, slope,
                     aspect, lats, lons, area):
    #load the data and get the inputs for the calculation 
    swdir_data=xr.open_dataset(input_dir_radiation, chunks={'lon': 100}) 
    swdif_data=xr.open_dataset(input_dif_radiation, chunks={'lon': 100}) 
    timepre = swdir_data["time"].values
    swdir_var=swdir_data["var22"]
    swdif_var=swdif_data["var23"]
    temp_data = xr.open_dataset(input_temp_data) 
    temp_var = temp_data["var11"].sel(lon=lons,lat=lats,method='nearest')
    #generate a xarray array to store the results
    swdir = swdir_var.sel(lon=lons,lat=lats,method='nearest')
    swdif = swdif_var.sel(lon=lons,lat=lats,method='nearest')
    Azimuth = pvlib.solarposition.get_solarposition(timepre, lats, lons).azimuth.values
    Zenith = pvlib.solarposition.get_solarposition(timepre, lats, lons).zenith.values
    AOI = pvlib.irradiance.aoi(slope, aspect, Zenith, Azimuth)
    diffuse = pvlib.irradiance.get_sky_diffuse(slope, aspect, Zenith, Azimuth, swdir, (swdir+swdif), swdif)
    reflected = pvlib.irradiance.get_ground_diffuse(slope,(swdir+swdif),0.25)
    inplane_ghi_var = pvlib.irradiance.poa_components(AOI,swdir,diffuse,reflected)["poa_global"]
    temp_air = temp_var-273.15 #rea6 temp and conversion to celsius
    panel_eff = 0.21          #pv panel efficiency, up to 21% nowadays
    temp_corr = -0.0045       #temperature correction factor, pv less efficient when higher temp
    reduction_factor = 0.035  #reduction factor due to installation type
    temp_nominal = 25   #nominal operating temperature, default 25 degrees C
    production = (inplane_ghi_var*panel_eff*(1+((temp_corr)*
                ((temp_air+(reduction_factor*inplane_ghi_var/area))
                -temp_nominal))))*area/1000
    production.plot()
    return production.values

#set the year
year = "2003" #further options are "2010" and "1995to2015"
input_dir_radiation = str("/media/luis_ws2/HDD2/CrossEnergy_deliverables/" 
                          + "basis_reanalysis_data/solar_radiation/"
                          + "SWDIRS_RAD.2D."+str(year)+".nc")
input_dif_radiation = str("/media/luis_ws2/HDD2/CrossEnergy_deliverables/"
                          +"basis_reanalysis_data/solar_radiation/"
                          +"/SWDIFDS_RAD.2D."+str(year)+".nc")
input_temp_data = str("/media/luis_ws2/HDD2/CrossEnergy_deliverables/"
                      + "basis_reanalysis_data/temperature/"
                      + "T_2M.2D."+str(year)+".nc")
#slope of the PV (0-90)
slope = 70
#aspect of the PV (starting in the North)
aspect = 180
lats = 50.0
lons = 5.0
area = 4.8 #area of pv panel, 4.8 sqm for aroung 1 kWp considering an efficiency of 21%
calc_pv_output(year, input_dir_radiation, input_dif_radiation, slope,
                     aspect, lats, lons, area)